# Exploring GPT-5: GPT-5 Free-Form Tool Call


In [ ]:
import os

from dotenv import load_dotenv

from autogen import ConversableAgent, LLMConfig
from autogen.agentchat import initiate_group_chat
from autogen.agentchat.group.patterns import AutoPattern

load_dotenv()

### Free‑Form Function Calling

 GPT‑5 can now send raw text payloads—such as Python scripts, SQL queries, or shell commands—directly to your custom tool using the new `"type": "custom"` tool interface. 
 
 Unlike classic structured function calls (which wrap arguments in JSON), the custom tool type gives you maximum flexibility for interacting with external runtimes, including:
 
 - Code execution sandboxes (Python, C++, Java, etc.)
 - SQL databases
 - Shell environments
 - Configuration generators
 
 **Note:** The custom tool type does *not* support parallel tool calling.
 
 This notebook demonstrates how to use GPT-5's free-form tool call capability to send and process raw text payloads.

**Use when structured JSON isn’t needed and raw text is more natural for the target tool.**


 ### Agent Setup


 In this section, we define the agents that will participate in the multi-agent chat. Each agent is configured with a specific role and behavior:

 - **code_agent**: Responsible for generating code based on the user's request.
 - **reviewer_agent**: Reviews the code produced by the code_agent and provides feedback.
 - **user**: Acts as the initiator of the conversation, providing tasks and interacting with the agents.

 We use the `ConversableAgent` class to create these agents, specifying their names, system messages, and interaction modes. The agents are configured to communicate using the GPT-5 model, and their behaviors are tailored to support free-form tool calls and collaborative workflows.


In [ ]:
llm_config = LLMConfig(
    model="gpt-5",
    api_key=os.getenv("OPENAI_API_KEY"),
    api_type="openai",
)

with llm_config:
    code_agent = ConversableAgent(
        name="code_agent",
        system_message="you are a coding agent. you need to code the task given to you",
        max_consecutive_auto_reply=5,
        human_input_mode="NEVER",
    )

    reviewer_agent = ConversableAgent(
        name="reviewer_agent",
        human_input_mode="NEVER",
        system_message="you are a reviewer agent. you need to review the code given to you",
    )

user = ConversableAgent(
    name="user",
    human_input_mode="ALWAYS",
    llm_config=llm_config,
)

### Register a free-form tool for the LLM by setting free_form=True

In [ ]:
@reviewer_agent.register_for_llm(description="This tool is used to review the code.", free_form=True)
@user.register_for_execution()
def review_the_code_tool(review: str):
    """
    This tool is used to review the code.
    Args:
        review: The review of the code.
    Returns:
        The review of the code.
    """
    return review

Execute the orchestration utilizing the free-form tool

In [ ]:
pattern = AutoPattern(
    initial_agent=code_agent,
    agents=[code_agent, reviewer_agent],
    user_agent=user,
    group_manager_args={"llm_config": llm_config},
)

result, context_variables, last_agent = initiate_group_chat(
    pattern=pattern,
    messages="write a python code to solve the 2 sums problem",
    max_rounds=5,
)